In [11]:
#available models

import pytorch_models
model_names = sorted(name for name in pytorch_models.__dict__
                     if name.islower() and not name.startswith("__")
                     and callable(pytorch_models.__dict__[name]))

print(model_names)

['bn', 'bnf_resnet50', 'bnz_resnet50', 'conv3x3', 'darknet_50', 'darknet_mini', 'darknet_mini2', 'darknet_mini3', 'darknet_small', 'dpn107', 'dpn131', 'dpn68', 'dpn92', 'dpn98', 'inceptionresnetv2', 'inceptionresnetv2_conc', 'inceptionv4', 'load', 'load_block17', 'load_block35', 'load_block8', 'load_conv2d', 'load_conv2d_nobn', 'load_linear', 'load_mixed_4a_7a', 'load_mixed_5', 'load_mixed_5b', 'load_mixed_6', 'load_mixed_6a', 'load_mixed_7', 'load_mixed_7a', 'pre_resnet101', 'pre_resnet152', 'pre_resnet18', 'pre_resnet34', 'pre_resnet50', 'resnet101', 'resnet152', 'resnet18', 'resnet34', 'resnet50', 'resnet50_3', 'resnext101', 'resnext152', 'resnext18', 'resnext34', 'resnext50', 'se_resnet_101', 'se_resnet_152', 'se_resnet_18', 'se_resnet_34', 'se_resnet_50', 'se_resnet_50_conc', 'se_resnext_101', 'se_resnext_152', 'se_resnext_50', 'test', 'test_block17', 'test_block35', 'test_block8', 'test_conv2d', 'test_conv2d_nobn', 'test_mixed_4a_7a', 'test_mixed_5b', 'test_mixed_6a', 'test_mixed

In [12]:
# minimal parameters

arch = "resnet18"

job_name = "resnet18_fp32"

save_dir = "xlog_180502_" + job_name

In [13]:
extra = " -b 128 --sz 256"

cmd = "python train2.py data --save-dir " + save_dir + " -a " + arch + extra

print(cmd)

python train2.py data --save-dir xlog_180502_resnet18_fp32 -a resnet18 -b 128 --sz 256


In [14]:
%%bash -s "{job_name}" "{cmd}" "{save_dir}" 
echo "#!/bin/bash
#SBATCH --job-name=train_$1
#SBATCH --output=train_$1.sbatch_out
#SBATCH --error=train_$1.sbatch_err
#SBATCH --time=35:00:00
#SBATCH --nodes=1
#SBATCH --partition=gpu2
#SBATCH --gres=gpu:1
#SBATCH --ntasks-per-node=8
#SBATCH --mem=32000

module load cuda/9.0
module load Anaconda3/5.0.1
source activate fastai

$2

cp checkpoint.pth.tar $3

mv $1.sbatch $3
mv train_$1.sbatch_out $3
mv train_$1.sbatch_err $3
" > $1.sbatch

cat $1.sbatch

#!/bin/bash
#SBATCH --job-name=train_resnet18_fp32
#SBATCH --output=train_resnet18_fp32.sbatch_out
#SBATCH --error=train_resnet18_fp32.sbatch_err
#SBATCH --time=35:00:00
#SBATCH --nodes=1
#SBATCH --partition=gpu2
#SBATCH --gres=gpu:1
#SBATCH --ntasks-per-node=8
#SBATCH --mem=32000

module load cuda/9.0
module load Anaconda3/5.0.1
source activate fastai

python train2.py data --save-dir xlog_180502_resnet18_fp32 -a resnet18 -b 128 --sz 256

cp checkpoint.pth.tar xlog_180502_resnet18_fp32

mv resnet18_fp32.sbatch xlog_180502_resnet18_fp32
mv train_resnet18_fp32.sbatch_out xlog_180502_resnet18_fp32
mv train_resnet18_fp32.sbatch_err xlog_180502_resnet18_fp32



In [10]:
str(3).zfill(3)

'003'

In [ ]:
def get_parser():
    parser = argparse.ArgumentParser(description='PyTorch ImageNet Training')
    parser.add_argument('data', metavar='DIR',
                        help='path to dataset')
    parser.add_argument('--save-dir', type=str, default=Path.cwd(),
                        help='Directory to save logs and models.')
    parser.add_argument('--arch', '-a', metavar='ARCH', default='resnet18',
                        choices=model_names,
                        help='model architecture: ' +
                        ' | '.join(model_names) +
                        ' (default: resnet18)')
    parser.add_argument('-j', '--workers', default=7, type=int, metavar='N',
                        help='number of data loading workers (default: 7)')
    parser.add_argument('--epochs', default=90, type=int, metavar='N',
                        help='number of total epochs to run')
    parser.add_argument('--start-epoch', default=0, type=int, metavar='N',
                        help='manual epoch number (useful on restarts)')
    parser.add_argument('-b', '--batch-size', default=256, type=int,
                        metavar='N', help='mini-batch size (default: 256)')
    parser.add_argument('--lr', '--learning-rate', default=0.1, type=float,
                        metavar='LR', help='initial learning rate')
    parser.add_argument('--momentum', default=0.9, type=float, metavar='M', help='momentum')
    parser.add_argument('--weight-decay', '--wd', default=1e-4, type=float,
                        metavar='W', help='weight decay (default: 1e-4)')
    parser.add_argument('--print-freq', '-p', default=100, type=int,
                        metavar='N', help='print frequency (default: 100)')
    parser.add_argument('--resume', default='', type=str, metavar='PATH',
                        help='path to latest checkpoint (default: none)')
    parser.add_argument('-e', '--evaluate', dest='evaluate', action='store_true',
                        help='evaluate model on validation set')
    parser.add_argument('--pretrained', dest='pretrained', action='store_true', help='use pre-trained model')
    parser.add_argument('--fp16', action='store_true', help='Run model fp16 mode.')
    parser.add_argument('--dp', action='store_true', help='Run model fp16 mode.')
    parser.add_argument('--sz',       default=224, type=int, help='Size of transformed image.')
    parser.add_argument('--decay-int', default=30, type=int, help='Decay LR by 10 every decay-int epochs')
    parser.add_argument('--loss-scale', type=float, default=1,
                        help='Loss scaling, positive power of 2 values can improve fp16 convergence.')
    parser.add_argument('--prof', dest='prof', action='store_true', help='Only run a few iters for profiling.')

    parser.add_argument('--dist-url', default='file://sync.file', type=str,
                        help='url used to set up distributed training')
    parser.add_argument('--dist-backend', default='nccl', type=str, help='distributed backend')

    parser.add_argument('--world-size', default=1, type=int,
                        help='Number of GPUs to use. Can either be manually set ' +
                        'or automatically set by using \'python -m multiproc\'.')
    parser.add_argument('--rank', default=0, type=int,
                        help='Used for multi-process training. Can either be manually set ' +
                        'or automatically set by using \'python -m multiproc\'.')
    
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every few epochs"""
    if   epoch<4 : lr = args.lr/(4-epoch)
    elif epoch<28: lr = args.lr/1
    elif epoch<47: lr = args.lr/10
    elif epoch<57: lr = args.lr/100
    else         : lr = args.lr/1000
    for param_group in optimizer.param_groups: param_group['lr'] = lr    

In [40]:
import pytorch_models
model = pytorch_models.__dict__['resnet50'](pretrained=True)

In [38]:
print(model)

<function resnet50_3 at 0x7f26201388c8>
